In [71]:
# ! pip install -U rank_bm25

In [72]:
from sentence_transformers import SentenceTransformer, util

In [73]:
import pandas as pd

from sentence_transformers import SentenceTransformer, CrossEncoder, util

from tqdm.autonotebook import tqdm
import torch

if not torch.cuda.is_available():
    print("No GPU found")

In [74]:

docs = pd.read_csv('quran_id_full.csv')
docs = docs['indoText']
docs

0       Dengan nama Allah Yang Maha Pengasih, Maha Pen...
1             Segala puji bagi Allah, Tuhan seluruh alam,
2                     Yang Maha Pengasih, Maha Penyayang,
3                                Pemilik hari pembalasan.
4       Hanya kepada Engkaulah kami menyembah dan hany...
                              ...                        
6231                                        Raja manusia,
6232                                    sembahan manusia,
6233     dari kejahatan (bisikan) setan yang bersembunyi,
6234    yang membisikkan (kejahatan) ke dalam dada man...
6235                    dari (golongan) jin dan manusia.”
Name: indoText, Length: 6236, dtype: object

In [81]:
query = "apa itu jalan yang lurus?" # 

In [82]:
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')
query_emb =  model.encode(query)
doc_emb =  model.encode(docs)
scores = util.dot_score(query_emb, doc_emb)[0].cpu()
doc_score_pairs = list(zip(docs, scores))
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
for doc, score in doc_score_pairs:
    print(score, doc)

tensor(0.7866) (yang berada) di atas jalan yang lurus,
tensor(0.7414) Sungguh Allah, Dia Tuhanku dan Tuhanmu, maka sembahlah Dia. Ini adalah jalan yang lurus.”
tensor(0.7324) Dan sesungguhnya engkau pasti telah menyeru mereka kepada jalan yang lurus.
tensor(0.7318) Tunjukilah kami jalan yang lurus,
tensor(0.7267) Dan hak Allah menerangkan jalan yang lurus, dan di antaranya ada (jalan) yang menyimpang. Dan jika Dia menghendaki, tentu Dia memberi petunjuk kamu semua (ke jalan yang benar).
tensor(0.7217) Sesungguhnya Allah itu Tuhanku dan Tuhanmu, karena itu sembahlah Dia. Inilah jalan yang lurus.”
tensor(0.7160) Kemudian dia menempuh suatu jalan (yang lain).
tensor(0.7147) Sungguh, Kami telah menunjukkan kepadanya jalan yang lurus; ada yang bersyukur dan ada pula yang kufur.
tensor(0.7135) Kemudian dia menempuh suatu jalan (yang lain lagi).
tensor(0.7095) Dan sesungguhnya orang-orang yang tidak beriman kepada akhirat benar-benar telah menyimpang jauh dari jalan (yang lurus).
tensor(0.707

In [83]:
docs.to_list()

['Dengan nama Allah Yang Maha Pengasih, Maha Penyayang.',
 'Segala puji bagi Allah, Tuhan seluruh alam,',
 'Yang Maha Pengasih, Maha Penyayang,',
 'Pemilik hari pembalasan.',
 'Hanya kepada Engkaulah kami menyembah dan hanya kepada Engkaulah kami mohon pertolongan.',
 'Tunjukilah kami jalan yang lurus,',
 '(yaitu) jalan orang-orang yang telah Engkau beri nikmat kepadanya; bukan (jalan) mereka yang dimurkai, dan bukan (pula jalan) mereka yang sesat.',
 'Alif Lam Mim.',
 "Kitab (Al-Qur'an) ini tidak ada keraguan padanya; petunjuk bagi mereka yang bertakwa,",
 '(yaitu) mereka yang beriman kepada yang gaib, melaksanakan salat, dan menginfakkan sebagian rezeki yang Kami berikan kepada mereka,',
 "dan mereka yang beriman kepada (Al-Qur'an) yang diturunkan kepadamu (Muhammad) dan (kitab-kitab) yang telah diturunkan sebelum engkau, dan mereka yakin akan adanya akhirat.",
 'Merekalah yang mendapat petunjuk dari Tuhannya, dan mereka itulah orang-orang yang beruntung.',
 'Sesungguhnya orang-orang

In [84]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

import numpy as np

def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

tokenized_corpus = []
for doc in docs:
    tokenized_corpus.append(bm25_tokenizer(doc))
tokenized_corpus

[['dengan', 'nama', 'allah', 'yang', 'maha', 'pengasih', 'maha', 'penyayang'],
 ['segala', 'puji', 'bagi', 'allah', 'tuhan', 'seluruh', 'alam'],
 ['yang', 'maha', 'pengasih', 'maha', 'penyayang'],
 ['pemilik', 'hari', 'pembalasan'],
 ['hanya',
  'kepada',
  'engkaulah',
  'kami',
  'menyembah',
  'dan',
  'hanya',
  'kepada',
  'engkaulah',
  'kami',
  'mohon',
  'pertolongan'],
 ['tunjukilah', 'kami', 'jalan', 'yang', 'lurus'],
 ['yaitu',
  'jalan',
  'orang-orang',
  'yang',
  'telah',
  'engkau',
  'beri',
  'nikmat',
  'kepadanya',
  'bukan',
  'jalan',
  'mereka',
  'yang',
  'dimurkai',
  'dan',
  'bukan',
  'pula',
  'jalan',
  'mereka',
  'yang',
  'sesat'],
 ['alif', 'lam', 'mim'],
 ['kitab',
  "al-qur'an",
  'ini',
  'tidak',
  'ada',
  'keraguan',
  'padanya',
  'petunjuk',
  'bagi',
  'mereka',
  'yang',
  'bertakwa'],
 ['yaitu',
  'mereka',
  'yang',
  'beriman',
  'kepada',
  'yang',
  'gaib',
  'melaksanakan',
  'salat',
  'dan',
  'menginfakkan',
  'sebagian',
  'rezeki

In [85]:
bm25 = BM25Okapi(tokenized_corpus)
bm25

In [87]:
print("Query:", query)

bm25_scores = bm25.get_scores(bm25_tokenizer(query))
top_n = np.argpartition(bm25_scores, -10)[-10:]
bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

print("Top-10 lexical search (BM25) hits")
for hit in bm25_hits[0:10]:
    print("\t{:.3f}\t{}".format(hit['score'], docs[hit['corpus_id']].replace("\n", " ")))

Query: apa itu jalan yang lurus?
Top-10 lexical search (BM25) hits
	15.078	Tunjukilah kami jalan yang lurus,
	14.951	(yang berada) di atas jalan yang lurus,
	14.237	dan Kami tunjukkan keduanya jalan yang lurus.
	13.484	dan pasti Kami tunjukkan kepada mereka jalan yang lurus.
	13.263	(yaitu) bagi siapa di antara kamu yang menghendaki menempuh jalan yang lurus.
	13.137	Dia (Allah) berfirman, “Ini adalah jalan yang lurus (menuju) kepada-Ku.”
	12.847	Katakanlah (Muhammad), “Sesungguhnya Tuhanku telah memberiku petunjuk ke jalan yang lurus, agama yang benar, agama Ibrahim yang lurus. Dia (Ibrahim) tidak termasuk orang-orang musyrik.”
	12.807	Dan sesungguhnya engkau pasti telah menyeru mereka kepada jalan yang lurus.
	12.578	Sungguh, Kami telah menunjukkan kepadanya jalan yang lurus; ada yang bersyukur dan ada pula yang kufur.
	12.493	dia mensyukuri nikmat-nikmat-Nya. Allah telah memilihnya dan menunjukinya ke jalan yang lurus.
